# Preparación de los Datos

In [56]:
import pandas as pd

In [57]:
data = pd.read_csv('../datasets/twitter_trump_data.csv')

### Convertir a DataFrame

In [58]:
df = pd.DataFrame(data)

In [59]:
df.head()

,text,retweet_count,favorite_count,Followers,Follower_Change,Num_Tweets,Year,Month,Week,Day,Hour
0,Judge Kavanaugh showed America exactly why I n...,85502,320158,54672474,21128,12,2018,9,39,27,22
1,https://t.co/9o5gZlJiTd,17391,77145,54651346,20751,13,2018,9,39,26,23
2,Join me this Saturday in Wheeling West Virgini...,14484,52957,54651346,20751,13,2018,9,39,26,20
3,Congressman Lee Zeldin is doing a fantastic jo...,14949,65494,54651346,20751,13,2018,9,39,26,17
4,China is actually placing propaganda ads in th...,19236,58671,54651346,20751,13,2018,9,39,26,17


### Renombrar las columnas

In [60]:
df = df.rename(columns={
    "Followers": "followers",
    "Follower_Change": "follower_change",
    "Num_Tweets": "tweets",
    "Year": "year",
    "Month": "month",
    "Week": "week",
    "Day": "day",
    "Hour": "hour"
})

### Valores nulos por variable

In [61]:
df = df[df != 0]

In [62]:
pd.isnull(df).sum()

text                 0
retweet_count        0
favorite_count       0
followers            0
follower_change      8
tweets             118
year                 0
month                0
week                 0
day                  0
hour               600
dtype: int64

### Resumen estadístico

- count: Número de elementos no nulos
- mean: Promedio
- std: Desviación estándar
- min: Valor mínimo
- 25%: Primer cuartil
- 50%: Segundo cuartil
- 75%: Tercer cuartil
- max: Valor máximo

In [63]:
 df.describe()

,retweet_count,favorite_count,followers,follower_change,tweets,year,month,week,day,hour
count,11759.000000,11759.000000,1.175900e+04,11751.000000,11641.000000,11759.000000,11759.000000,11759.000000,11759.000000,11159.000000
mean,10542.179012,40495.229101,2.002753e+07,33561.168581,16.235547,2016.246875,7.017604,28.794200,16.127307,13.375213
std,12463.598542,45900.885681,1.858377e+07,34112.552562,14.563559,1.066680,3.237644,14.067874,8.823738,6.600350
min,6.000000,34.000000,2.957264e+06,-329029.000000,-29.000000,2015.000000,1.000000,1.000000,1.000000,1.000000
25%,1516.000000,3727.000000,5.033780e+06,12735.000000,7.000000,2015.000000,5.000000,18.000000,8.000000,10.000000
50%,6806.000000,19503.000000,9.941948e+06,22188.000000,13.000000,2016.000000,7.000000,30.000000,17.000000,14.000000
75%,16433.000000,70692.000000,3.780345e+07,44629.000000,20.000000,2017.000000,10.000000,41.000000,24.000000,19.000000
max,369530.000000,633253.000000,5.467247e+07,801107.000000,98.000000,2018.000000,12.000000,53.000000,31.000000,23.000000
